# Statement Extractor
> Statement extraction through claim detection

This Notebook guides you through the *Statement Extractor* module.

In [1]:
from src.utils import pdf_parser
from src import segmenter
from config import SAMPLE_PDF_PATH

import pandas as pd

## 0. Load data

In [2]:
text = pdf_parser.pdf_to_text(SAMPLE_PDF_PATH)  # load all text from pdf

In [3]:
parse = pdf_parser.parse_text(text)  # parse pdf with specific document parser

## 1. Create segments

In [4]:
title = parse["title"]
date = parse["date"]
passages = parse["passages"]  # this document has natural passages

In [5]:
doc = []  # create empty doc

In [6]:
passage_id = 0

for passage in passages:
    segments = segmenter.split_segments(passage.get("text"))
    
    for segment in segments:
        segment["passage_id"] = passage_id
        doc.append(segment)
    
    passage_id += 1

### 1.1 Load to table

In [7]:
doc_table = pd.DataFrame(doc)

In [9]:
doc_table.head()

,segment_id,sentence,passage_id
0,0,Guten Mittag liebe Journalistinnen und Journal...,0
1,1,Herzlich willkommen hier zu unserem virtuellen...,0
2,2,"Mein Name ist Bastian Zimmermann, und ich bin ...",0
3,3,"Ich freue mich, heute auch unsere drei Experte...",0
4,0,"Man hört immer wieder, wie wichtig Quantentech...",0


### 1.2 Add Metadata

In [10]:
doc_table["title"] = title
doc_table["date"] = date

In [12]:
doc_table["speaker"] = None
doc_table["timestamp"] = None

for idx, passage in enumerate(passages):
    speaker = passage.get("speaker")
    timestamp = passage.get("timestamp")
    
    doc_table['speaker'].loc[doc_table['passage_id'] == idx] = speaker
    doc_table['timestamp'].loc[doc_table['passage_id'] == idx] = timestamp


/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
doc_table.head()

,segment_id,sentence,passage_id,title,date,speaker,timestamp
0,0,Guten Mittag liebe Journalistinnen und Journal...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
1,1,Herzlich willkommen hier zu unserem virtuellen...,0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
2,2,"Mein Name ist Bastian Zimmermann, und ich bin ...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
3,3,"Ich freue mich, heute auch unsere drei Experte...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]
4,0,"Man hört immer wieder, wie wichtig Quantentech...",0,„Steht der Quantenrechner vor der Tür? Forschu...,12.04.2021,Moderator,[00:00:00]


## 2. (Detect main concept)

## 3. Detect claim sentences

## 4. Return statements